![CH3-ADS.png](.\Media\CH3-ADS.png)

# <span style="color:rgb(204, 85, 0);">Get NUMA information</span>

<span style="font-size: 14px;">Modern processors have multiple cores per socket. Each socket is represented, usually, as a single NUMA node. The SQL Server database engine partitions various internal structures and partitions service threads per NUMA node. With processors containing 10 or more cores per socket, using software NUMA to split hardware NUMA nodes generally increases scalability and performance. Prior to SQL Server 2014 (12.x) SP2, software-based NUMA (soft-NUMA) required you to edit the registry to add a node configuration affinity mask, and was configured at the host level, rather than per instance. Starting with SQL Server 2014 (12.x) SP2 and SQL Server 2016 (13.x), soft-NUMA is configured automatically at the database-instance level when the SQL Server Database Engine service starts.</span>

With SQL Server 2016 (13.x), whenever the SQL Server Database Engine detects more than eight physical cores per NUMA node or socket at startup, soft-NUMA nodes are created automatically by default. Hyper-threaded processor cores are not differentiated when counting physical cores in a node. When the detected number of physical cores is more than eight per socket, the SQL Server Database Engine creates soft-NUMA nodes that ideally contain eight cores, but can go down to five or up to nine logical cores per node. The size of the hardware node can be limited by a CPU affinity mask. The number of NUMA nodes never exceeds the maximum number of supported NUMA nodes. 

  

Changing CPU Affinities should only be performed by an experienced DBA on a case by case basis.  [Understanding Non-uniform Memory Access | Microsoft Docs](https://docs.microsoft.com/en-us/previous-versions/sql/sql-server-2008-r2/ms178144(v=sql.105)) and [NUMA Scenarios | Microsoft Docs](https://docs.microsoft.com/en-us/previous-versions/sql/sql-server-2008-r2/ms345345(v=sql.105))

  

### <span style="color:rgb(0, 204, 153);">Run the Code block below</span>

1. Click the run icon below
2. If ADS prompts you for a connection, enter the correct SQL Server and authentication account
3. View the results of the query by scrolling down to the results set

In [ ]:
SET NOCOUNT ON;
SET ANSI_WARNINGS ON;
SET QUOTED_IDENTIFIER ON;

DECLARE @cpucount int, @numa int, @affined_cpus int, @affinitymask NVARCHAR(64), @affinity64mask NVARCHAR(1024)--, @cpuover32 int
DECLARE @i int, @cpuaffin VARCHAR(300), @cpuaffin_fixed VARCHAR(300)

SELECT @numa = COUNT(DISTINCT parent_node_id) FROM sys.dm_os_schedulers WHERE scheduler_id < 255 AND parent_node_id < 64;
SELECT @cpucount = COUNT(cpu_id) FROM sys.dm_os_schedulers WHERE scheduler_id < 255 AND parent_node_id < 64

;WITH bits AS 
(SELECT 7 AS N, 128 AS E UNION ALL SELECT 6, 64 UNION ALL 
SELECT 5, 32 UNION ALL SELECT 4, 16 UNION ALL SELECT 3, 8 UNION ALL 
SELECT 2, 4 UNION ALL SELECT 1, 2 UNION ALL SELECT 0, 1), 
bytes AS 
(SELECT 1 M UNION ALL SELECT 2 UNION ALL SELECT 3 UNION ALL 
SELECT 4 UNION ALL SELECT 5 UNION ALL SELECT 6 UNION ALL 
SELECT 7 UNION ALL SELECT 8 UNION ALL SELECT 9)
-- CPU Affinity is shown highest to lowest CPU ID
SELECT @affinitymask = CASE WHEN [value] = 0 THEN REPLICATE('1', @cpucount)
	ELSE RIGHT((SELECT ((CONVERT(tinyint, SUBSTRING(CONVERT(binary(9), [value]), M, 1)) & E) / E) AS [text()] 
		FROM bits CROSS JOIN bytes
		ORDER BY M, N DESC
		FOR XML PATH('')), @cpucount) END
FROM sys.configurations (NOLOCK)
WHERE name = 'affinity mask';

IF @cpucount > 32
BEGIN
	;WITH bits AS 
	(SELECT 7 AS N, 128 AS E UNION ALL SELECT 6, 64 UNION ALL 
	SELECT 5, 32 UNION ALL SELECT 4, 16 UNION ALL SELECT 3, 8 UNION ALL 
	SELECT 2, 4 UNION ALL SELECT 1, 2 UNION ALL SELECT 0, 1), 
	bytes AS 
	(SELECT 1 M UNION ALL SELECT 2 UNION ALL SELECT 3 UNION ALL 
	SELECT 4 UNION ALL SELECT 5 UNION ALL SELECT 6 UNION ALL 
	SELECT 7 UNION ALL SELECT 8 UNION ALL SELECT 9)
	-- CPU Affinity is shown highest to lowest CPU ID
	SELECT @affinity64mask = CASE WHEN [value] = 0 THEN REPLICATE('1', @cpucount)
		ELSE RIGHT((SELECT ((CONVERT(tinyint, SUBSTRING(CONVERT(binary(9), [value]), M, 1)) & E) / E) AS [text()] 
			FROM bits CROSS JOIN bytes
			ORDER BY M, N DESC
			FOR XML PATH('')), @cpucount) END
	FROM sys.configurations (NOLOCK)
	WHERE name = 'affinity64 mask';
END;

/*
IF @cpucount > 32
SELECT @cpuover32 = ABS(LEN(@affinity64mask) - (@cpucount-32))

SELECT @cpuaffin = CASE WHEN @cpucount > 32 THEN REVERSE(LEFT(REVERSE(@affinity64mask),@cpuover32)) + RIGHT(@affinitymask,32) ELSE RIGHT(@affinitymask,@cpucount) END
*/

SELECT @cpuaffin = CASE WHEN @cpucount > 32 THEN @affinity64mask ELSE @affinitymask END

SET @cpuaffin_fixed = @cpuaffin
SET @i = CEILING(@cpucount*1.00/@numa) + 1
WHILE @i < @cpucount + @numa
BEGIN
	IF (@cpucount + @numa) - @i >= CEILING(@cpucount*1.00/@numa)
	BEGIN
		SELECT @cpuaffin_fixed = STUFF(@cpuaffin_fixed, @i, 1, '_' + SUBSTRING(@cpuaffin_fixed, @i, 1))
	END
	ELSE
	BEGIN
		SELECT @cpuaffin_fixed = STUFF(@cpuaffin_fixed, @i, CEILING(@cpucount*1.00/@numa), SUBSTRING(@cpuaffin_fixed, @i, CEILING(@cpucount*1.00/@numa)))
	END

	SET @i = @i + CEILING(@cpucount*1.00/@numa) + 1
END;

IF @numa > 1
BEGIN
	WITH ncpuCTE (ncpus) AS (SELECT COUNT(cpu_id) AS ncpus from sys.dm_os_schedulers WHERE is_online = 1 AND scheduler_id < 255 AND parent_node_id < 64 GROUP BY parent_node_id, is_online HAVING COUNT(cpu_id) = 1),
	cpuCTE (node, afin) AS (SELECT DISTINCT(parent_node_id), is_online FROM sys.dm_os_schedulers WHERE scheduler_id < 255 AND parent_node_id < 64 GROUP BY parent_node_id, is_online)
	SELECT 'Processor_checks' AS [Category], 'Affinity_NUMA' AS [Check],
		CASE WHEN (SELECT COUNT(*) FROM ncpuCTE) > 0 THEN 'WARNING: Current NUMA configuration is not recommended. At least one node has a single assigned CPU' 
			WHEN (SELECT COUNT(DISTINCT(node)) FROM cpuCTE WHERE afin = 0 AND node NOT IN (SELECT DISTINCT(node) FROM cpuCTE WHERE afin = 1)) > 0 THEN 'WARNING: Current NUMA configuration is not recommended. At least one node does not have assigned CPUs' 
			ELSE 'OK' END AS [Deviation]
	FROM sys.dm_os_sys_info (NOLOCK) 
	OPTION (RECOMPILE);
	
	SELECT 'Processor_checks' AS [Category], 'Affinity_NUMA' AS [Information], cpu_count AS [Logical_CPU_Count], 
		(SELECT COUNT(DISTINCT parent_node_id) FROM sys.dm_os_schedulers WHERE scheduler_id < 255 AND parent_node_id < 64) AS [NUMA_Nodes],
		-- Processor Affinity is shown highest to lowest CPU ID
		@cpuaffin_fixed AS Affinity_Mask_Bitmask
	FROM sys.dm_os_sys_info (NOLOCK) 
	OPTION (RECOMPILE);
END
ELSE
BEGIN
	SELECT 'Processor_checks' AS [Category], 'Affinity_NUMA' AS [Check], 'Not NUMA' AS [Deviation]
	FROM sys.dm_os_sys_info (NOLOCK)
	OPTION (RECOMPILE);
END;